In [1]:
import rebound
import reboundx
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

In [2]:
planet_mass = 5 * u.M_earth.to(u.M_sun)
planet_ecc = 0.01
planet_radius = 2.5 * u.R_earth.to(u.AU)
planet_mass, planet_radius

(1.5017446744253966e-05, 0.00010658741281135093)

In [6]:
# set up rebound
sim = rebound.Simulation()
sim.G = 1.
sim.add(m=1., hash="star")
sim.add(m=planet_mass, a=0.17, 
        e=planet_ecc, r=planet_radius, hash="p1")
sim.add(m=planet_mass, a=0.23,
        e=planet_ecc, r=planet_radius, hash="p2")
sim.move_to_com()

# using WHFast with planet period = planet period / 12.134
ps = sim.particles
sim.integrator = "whfast"
sim.dt = ps[1].P / 12.1234

# add migration forces with characteristic timescales
rebx = reboundx.Extras(sim)
mof = rebx.load_force("modify_orbits_forces")
τ1 = 5. * u.Myr.to(u.yr) * (2 * np.pi)
τ2 = τ1 / 1.1
ps[1].params["tau_a"] = -τ1
ps[2].params["tau_a"] = -τ2
rebx.add_force(mof)

In [7]:
# integrate until 4 millions years
tmax = 4e6
N_outputs = 100

# set up arrays to store orbital elements
# 2 pi factors are to convert from yr to yr/2pi units
t_arr = np.linspace(0., tmax * (2*np.pi), N_outputs)

# storing semimajor axis, eccentricity, and inclination
a = np.zeros((N_outputs,2))
ecc = np.zeros((N_outputs,2))
inc = np.zeros((N_outputs,2))

for i, t in enumerate(t_arr):
    sim.integrate(t, exact_finish_time=1)
    
    # turn off migration after 2 million yrs (following Fig. 1, ML19)
    if t >= 2e6 and np.isfinite(ps[1].params["tau_a"]):
        ps[1].params["tau_a"] = np.inf
        ps[2].params["tau_a"] = np.inf
    
    ps = sim.particles
    a[i] = [ps[1].a, ps[2].a]
    ecc[i] = [ps[1].e, ps[2].e]
    inc[i] = [ps[1].inc, ps[2].inc]

0.0 0.0
253866.073017357 0.0101010101010101
507732.146034714 0.0202020202020202
761598.219052071 0.0303030303030303


KeyboardInterrupt: 

In [ ]:
plt.plot(t_arr / (2*np.pi), a[:,0])
plt.plot(t_arr / (2*np.pi), a[:,1])

# plt.ylim([0., 0.25])

In [ ]:
plt.plot(t_arr / (2*np.pi), ecc[:,0])
plt.plot(t_arr / (2*np.pi), ecc[:,1])

Now migration + obliquity evolution + equilibrium tides

intro to equilibrium tides

In [ ]:
# set up rebound
sim = rebound.Simulation()
sim.G = 1.
sim.add(m=1., hash="star")
sim.add(m=planet_mass, a=0.17, 
        e=planet_ecc, r=planet_radius, hash="p1")
sim.add(m=planet_mass, a=0.23,
        e=planet_ecc, r=planet_radius, hash="p2")
sim.move_to_com()

# using WHFast with planet period = planet period / 12.134
ps = sim.particles
sim.integrator = "whfast"
sim.dt = ps[1].P / 12.1234

# add migration forces with characteristic timescales
rebx = reboundx.Extras(sim)
mof = rebx.load_force("modify_orbits_forces")
τ1 = 5. * u.Myr.to(u.yr) * (2 * np.pi)
τ2 = τ1 / 1.1
ps[1].params["tau_a"] = -τ1
ps[2].params["tau_a"] = -τ2
rebx.add_force(mof)

sf = rebx.load_force("tides_spin")
rebx.add_force(sf)

In [ ]:
# Star parameters
# Stellar rotation period = 20 days
C_star = 0.07 # normalized moment of inertia
ps[0].params['Omega'] = rebound.spherical_to_xyz(magnitude=2*np.pi/(20/365), theta=0, phi=0)
ps[0].params['I'] = C_star * ps[0].m * ps[0].r**2

# Planet parameters
C_planet = 0.25 # normalized moment of inertia
Prot_planet1 = 5 # days
initial_obliquity = 1 # degrees
ps[1].params['Omega'] = rebound.spherical_to_xyz(magnitude=2*np.pi/(Prot_planet1/365), theta=np.radians(initial_obliquity), phi=0)
ps[1].params['I'] = C_planet * ps[1].m * ps[1].r**2

Prot_planet2 = 3 #days
ps[1].params['Omega'] = rebound.spherical_to_xyz(magnitude=2*np.pi/(Prot_planet2/365), theta=np.radians(initial_obliquity), phi=0)
ps[1].params['I'] = C_planet * ps[1].m * ps[1].r**2

In [ ]:
# initialize spin ODEs
rebx.initialize_spin_ode(sf)

In [ ]:
# Set k2 Love numbers so planets feel tides and deform from perfect spheres
ps[0].params['k2'] = 0.1
ps[1].params['k2'] = 0.4
ps[1].params['k2'] = 0.4